<a href="https://colab.research.google.com/github/Enzo-Damico/projet_MODIA/blob/master/simple_reco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading and pre-processing of data

In [1]:
from google.colab import drive
drive.mount('/content/drive')
path = "drive/MyDrive/Projet/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import zipfile

#df_interactions_test = pd.read_csv(path + "interactions_test.csv")
#df_interactions_train = pd.read_csv(path + "interactions_train.csv.zip")
df_RAW_interactions = pd.read_csv(path + "RAW_interactions.csv.zip")
df_RAW_recipes = pd.read_csv(path + "RAW_recipes.csv.zip")
print(df_RAW_interactions.shape)
print(df_RAW_recipes.shape)

(1132367, 5)
(231637, 12)


In [4]:
import ast

recipes_nvote = df_RAW_interactions["recipe_id"].value_counts()
rating = df_RAW_interactions[["recipe_id", "rating"]].groupby(["recipe_id"]).mean()
df_RAW_recipes["vote_count"] = df_RAW_recipes["id"].apply(lambda x: recipes_nvote.loc[x])
df_RAW_recipes["vote_mean"] = df_RAW_recipes["id"].apply(lambda x: rating.loc[x])
df_RAW_recipes["ingredients"] = df_RAW_recipes["ingredients"].apply(lambda s: ast.literal_eval(s)) 
df_RAW_recipes["steps"] = df_RAW_recipes["steps"].apply(lambda s: s.strip('[]'))

In [5]:
m = 100
c = df_RAW_recipes.vote_mean.mean()

def weighted_score(x):
    v = x['vote_count']
    r = x['vote_mean']
    return (v/(v+m) * r) + (m/(m+v) * c)

df_RAW_recipes["vote_w"] = df_RAW_recipes[["vote_count", "vote_mean"]].apply(lambda x: weighted_score(x), axis=1)
df_RAW_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,vote_count,vote_mean,vote_w
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"'make a choice and proceed with recipe', 'depe...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7,3,5.0,4.365287
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"'preheat oven to 425 degrees f', 'press dough ...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6,4,3.5,4.313698
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"'brown ground beef in large pot', 'add chopped...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13,1,4.0,4.342818
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,'place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...",11,2,4.5,4.349261
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,"'mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal...",8,1,5.0,4.352719


In [6]:
for dishes in df_RAW_recipes.nlargest(10, "vote_w")["name"]:
  print(dishes)

mexican stack up  rsc
kittencal s italian melt in your mouth meatballs
yummy crunchy apple pie
wholly guacamole
roasted cauliflower   16 roasted cloves of garlic
oven fried chicken chimichangas
kittencal s extreme chocolate brownies
brown sugar bundt cake
kittencal s famous caesar salad
crumb topped banana muffins


 # 1. Recommendation by popularity

## Best recipes by ingredients

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

subset = df_RAW_recipes[df_RAW_recipes['vote_count'] > 10].reset_index()

mlb = MultiLabelBinarizer()

ingredients_df = pd.DataFrame(mlb.fit_transform(subset['ingredients'].fillna('[]')),columns=mlb.classes_, index=subset.index)
ingredients_df.head()

,1% fat buttermilk,1% fat cottage cheese,1% low-fat milk,10-inch flour tortillas,10-minute herb stuffing mix,10-minute success rice,100 proof vodka,12-inch flour tortillas,12-inch pizza crust,15 bean soup mix,...,your favorite barbecue rub,your favorite sauce,yucca root,yukon gold potato,yukon gold potatoes,za'atar spice mix,ziploc bag,ziploc bags,ziti pasta,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
def reco_pop(ingredients):

  recettes = subset
  for ingredient in ingredients:
    recettes = recettes.loc[ingredients_df[ingredient]==1]

  recettes = recettes.nlargest(5, "vote_w")
  return recettes

recettes = reco_pop(["poblano pepper"])
recettes

,index,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,vote_count,vote_mean,vote_w
11346,134684,mexican stack up rsc,486261,75,226863,2012-09-01,"['time-to-make', 'course', 'main-ingredient', ...","[793.0, 83.0, 10.0, 34.0, 87.0, 131.0, 11.0]",28,"'preheat broiler to high broil', 'set the rack...","ready, set, cook! reynolds wrap contest entry....","[poblano pepper, reynolds wrap foil, olive oil...",22,217,4.990783,4.787459
8047,97045,green apple salsa the world s greatest by far,191845,30,368270,2006-10-23,"['30-minutes-or-less', 'time-to-make', 'course...","[119.2, 0.0, 92.0, 0.0, 2.0, 0.0, 10.0]",13,'squeeze as much juice and pulp as possible ou...,i had to come up with an apple salsa recipe in...,"[lemon, lime, honey, granny smith apples, whit...",7,14,4.500000,4.365128
704,8656,arroz con pollo,321647,75,74281,2008-08-27,"['time-to-make', 'course', 'cuisine', 'prepara...","[851.6, 46.0, 36.0, 182.0, 66.0, 32.0, 35.0]",15,'mix together the adobo seasoning in a small b...,i could not stop thinking about the arroz con ...,"[kosher salt, granulated garlic, ground cumin,...",23,12,4.250000,4.335934
4978,61815,cream of poblano soup,56270,55,65720,2003-03-12,"['60-minutes-or-less', 'time-to-make', 'course...","[160.8, 17.0, 3.0, 21.0, 13.0, 25.0, 2.0]",15,"'cut three tortillas into ninths', 'place in f...",poblano peppers are a mild mexican pepper and ...,"[corn tortillas, flour, chili powder, ground c...",15,14,4.214286,4.330040


# 2. Content based recommender systems

## According to description

In [9]:
from nltk import word_tokenize          
from nltk.stem import SnowballStemmer
import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
# Download stopwords list

stop_words = set(stopwords.words('english')) 

# Interface lemma tokenizer from nltk with sklearn
class StemTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.stemmer = SnowballStemmer('english')
    def __call__(self, doc):
        return [self.stemmer.stem(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]

tokenizer=StemTokenizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction.text import TfidfVectorizer


# Lemmatize the stop words
token_stop = tokenizer(' '.join(stop_words))

# Create TF-idf model
tfidf = TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, max_features=4000)

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(subset['description'].fillna(''))

cosine_sim = cosine_distances(tfidf_matrix, tfidf_matrix)

In [11]:
indices = pd.Series(subset.index, index=subset['name'])
recipes_name = subset["name"]

def reco_by_description(recipe_name, sim_matrix):
  idx = indices[recipe_name]
  print(f'original: {recipe_name}')
  recos = sim_matrix[idx].argsort()[1:6]
  recos = recipes_name.iloc[recos]

  print(recos)

recettes = reco_by_description("cream of poblano soup", cosine_sim)
recettes


original: never weep  whipped cream
12723    overnight french toast casserole
620                         apple pudding
3453                 cherry oatmeal crisp
16866     tater dipped oven fried chicken
16332       strawberry cream cake filling
Name: name, dtype: object


## According to steps

In [15]:
tfidf_matrix = tfidf.fit_transform(subset['steps'].fillna(''))
cosine_sim = cosine_distances(tfidf_matrix, tfidf_matrix)

def reco_by_steps(recipe_name, sim_matrix):
  idx = indices[recipe_name]
  print(f'original: {recipe_name}')
  recos = sim_matrix[idx].argsort()[1:6]
  recos = recipes_name.iloc[recos]

  print(recos)

recettes = reco_by_description("apple pudding", cosine_sim)
recettes

original: apple pudding
79                 1 2 3 apple crisp
11577               mom s apple tart
593                   apple crisp ii
634           apple upside down cake
2805     butterscotch apple crisp  2
Name: name, dtype: object
